Nome: Pietro Del Corona


PROBLEMA 1

Construir um algoritmo que implemente o Minimax para um jogo da velha.

O algoritmo Minimax cria uma árvore de busca que representa todas as possíveis sequências de jogadas para o jogo da velha, considerando todas as escolhas alternativas para ambos os jogadores, até um certo nível de profundidade (geralmente até um 

Estado terminal: D = Derrota, V = Vitória, E = Empate).

As pontuações são propagadas de baixo para cima na árvore, com os nós MAX (x) escolhendo o valor máximo entre seus filhos e os nós MIN (o) escolhendo o valor mínimo entre seus filhos. 

Os jogadores são alternadamente designados como "MAX (x)" e "MIN (o)". O MAX tenta maximizar sua pontuação (geralmente, vitória é +1 e derrota é -1), enquanto o MIN tenta minimizar a pontuação.

Final: Quando a raiz da árvore é alcançada, o MAX escolhe a jogada que leva ao filho com a maior pontuação, pois esse é o movimento que maximiza suas chances de ganhar.

Construa uma estrutura de dados mínima que permita jogar o Jogo da Velha, implementando o Minimax padrão, onde Max é o X, e você é o O. Iniciando o jogo com o MAX (x).

In [1]:
import math
import random
import time

In [2]:
class Jogador():
    def __init__(self, letra):
        self.letra = letra




class JogadorHumano(Jogador):
    def __init__(self, letra):
        super().__init__(letra)

    def fazer_jogada(self, jogo):
        quadrado_valido = False
        valor = None
        while not quadrado_valido:
            quadrado = input(self.letra + ' é a sua vez. Insira a jogada (0-9): ')
            try:
                valor = int(quadrado)
                if valor not in jogo.movimentos_disponiveis():
                    raise ValueError
                quadrado_valido = True
            except ValueError:
                print('Quadrado inválido. Tente novamente.')
        return valor


class JogadorComputadorInteligente(Jogador):
    def __init__(self, letra):
        super().__init__(letra)

    def fazer_jogada(self, jogo):
        if len(jogo.movimentos_disponiveis()) == 9:
            quadrado = random.choice(jogo.movimentos_disponiveis())
        else:
            quadrado = self.minimax(jogo, self.letra)['posicao']
        return quadrado

    def minimax(self, estado, jogador):
        jogador_max = self.letra  
        outro_jogador = 'O' if jogador == 'X' else 'X'

        
        if estado.vencedor_atual == outro_jogador:
            return {'posicao': None, 'pontuacao': 1 * (estado.num_quadrados_vazios() + 1) if outro_jogador == jogador_max else -1 * (
                        estado.num_quadrados_vazios() + 1)}
        elif not estado.quadrados_vazios():
            return {'posicao': None, 'pontuacao': 0}

        if jogador == jogador_max:
            melhor = {'posicao': None, 'pontuacao': -math.inf}  
        else:
            melhor = {'posicao': None, 'pontuacao': math.inf}  

        for jogada_possivel in estado.movimentos_disponiveis():
            estado.fazer_jogada(jogada_possivel, jogador)
            sim_pontuacao = self.minimax(estado, outro_jogador)  

            
            estado.tabuleiro[jogada_possivel] = ' '
            estado.vencedor_atual = None
            sim_pontuacao['posicao'] = jogada_possivel  

            if jogador == jogador_max:  
                if sim_pontuacao['pontuacao'] > melhor['pontuacao']:
                    melhor = sim_pontuacao
            else:
                if sim_pontuacao['pontuacao'] < melhor['pontuacao']:
                    melhor = sim_pontuacao
        return melhor

In [3]:
class JogoDaVelha():
    def __init__(self):
        self.tabuleiro = self.criar_tabuleiro()
        self.vencedor_atual = None

    def criar_tabuleiro(self):
        return [' ' for _ in range(9)]

    def imprimir_tabuleiro(self):
        for linha in [self.tabuleiro[i*3:(i+1) * 3] for i in range(3)]:
            print('| ' + ' | '.join(linha) + ' |')

    
    def imprimir_numeros_tabuleiro(self):
        numeros_tabuleiro = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for linha in numeros_tabuleiro:
            print('| ' + ' | '.join(linha) + ' |')

    def fazer_jogada(self, quadrado, letra):
        if self.tabuleiro[quadrado] == ' ':
            self.tabuleiro[quadrado] = letra
            if self.verificar_vencedor(quadrado, letra):
                self.vencedor_atual = letra
            return True
        return False

    def verificar_vencedor(self, quadrado, letra):
        
        linha_ind = math.floor(quadrado / 3)
        linha = self.tabuleiro[linha_ind*3:(linha_ind+1)*3]
        if all([s == letra for s in linha]):
            return True
        coluna_ind = quadrado % 3
        coluna = [self.tabuleiro[coluna_ind+i*3] for i in range(3)]
        if all([s == letra for s in coluna]):
            return True
        if quadrado % 2 == 0:
            diagonal1 = [self.tabuleiro[i] for i in [0, 4, 8]]
            if all([s == letra for s in diagonal1]):
                return True
            diagonal2 = [self.tabuleiro[i] for i in [2, 4, 6]]
            if all([s == letra for s in diagonal2]):
                return True
        return False

    def quadrados_vazios(self):
        return ' ' in self.tabuleiro

    def num_quadrados_vazios(self):
        return self.tabuleiro.count(' ')

    def movimentos_disponiveis(self):
        return [i for i, x in enumerate(self.tabuleiro) if x == " "]

def jogar(jogo, jogador_x, jogador_o, imprimir_jogo=True):
    if imprimir_jogo:
        jogo.imprimir_numeros_tabuleiro()
    letra = 'X'
    while jogo.quadrados_vazios():
        if letra == 'O':
            quadrado = jogador_o.fazer_jogada(jogo)
        else:
            quadrado = jogador_x.fazer_jogada(jogo)
        if jogo.fazer_jogada(quadrado, letra):
            if imprimir_jogo:
                print(letra + ' faz uma jogada no quadrado {}'.format(quadrado))
                jogo.imprimir_tabuleiro()
                print('')
            if jogo.vencedor_atual:
                if imprimir_jogo:
                    print(letra + ' vence!')
                return letra  
            letra = 'O' if letra == 'X' else 'X'  
        time.sleep(.8)
    if imprimir_jogo:
        print('É um empate!')

if __name__ == '__main__':
    jogador_x = JogadorComputadorInteligente('X')
    jogador_o = JogadorHumano('O')
    jogo = JogoDaVelha()
    jogar(jogo, jogador_x, jogador_o, imprimir_jogo=True)

| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |
X faz uma jogada no quadrado 3
|   |   |   |
| X |   |   |
|   |   |   |

O faz uma jogada no quadrado 5
|   |   |   |
| X |   | O |
|   |   |   |

X faz uma jogada no quadrado 0
| X |   |   |
| X |   | O |
|   |   |   |

O faz uma jogada no quadrado 7
| X |   |   |
| X |   | O |
|   | O |   |

X faz uma jogada no quadrado 6
| X |   |   |
| X |   | O |
| X | O |   |

X vence!
